<a href="https://colab.research.google.com/github/magenta/mt3/blob/main/mt3/colab/music_transcription_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Transcription with Transformers

This notebook is an interactive demo of a few [music transcription models](g.co/magenta/mt3) created by Google's [Magenta](g.co/magenta) team.  You can upload audio and have one of our models automatically transcribe it.

<img src="https://magenta.tensorflow.org/assets/transcription-with-transformers/architecture_diagram.png" alt="Transformer-based transcription architecture">

The notebook supports two pre-trained models:
1. the piano transcription model from [our ISMIR 2021 paper](https://archives.ismir.net/ismir2021/paper/000030.pdf)
1. the multi-instrument transcription model from [our ICLR 2022 paper](https://openreview.net/pdf?id=iMSjopcOn0p)

**Caveat**: neither model is trained on singing.  If you upload audio with vocals, you will likely get weird results.  Multi-instrument transcription is still not a completely-solved problem and so you may get weird results regardless.

In any case, we hope you have fun transcribing!  Feel free to tweet any interesting output at [@GoogleMagenta](https://twitter.com/googlemagenta)...

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each cell to execute the cell
* In the __Load Model__ cell, choose either `ismir2021` for piano transcription or `mt3` for multi-instrument transcription
* In the __Upload Audio__ cell, choose an MP3 or WAV file from your computer when prompted
* Transcribe the audio using the __Transcribe Audio__ cell (it may take a few minutes depending on the length of the audio)

---

This notebook sends basic usage data to Google Analytics.  For more information, see [Google's privacy policy](https://policies.google.com/privacy).


<a href="https://colab.research.google.com/github/magenta/mt3/blob/main/mt3/colab/music_transcription_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#音乐转录与变形金刚

这个笔记本是由Google的[magenta](g.co/magenta)团队创建的几个[音乐转录模型](g.co/magenta/mt3)的交互式演示。你可以上传音频，让我们的模型自动转录。

<img src="https://magenta.tensorflow.org/assets/transcription-with-transformers/architecture_diagram.png" alt="基于转换器的转录架构">

该笔记本支持两种预训练模型:
1. 来自[我们的ISMIR 2021论文]的钢琴转录模型(https://archives.ismir.net/ismir2021/paper/000030.pdf)
1. 来自[我们的ICLR 2022论文]的多乐器转录模型(https://openreview.net/pdf?id=iMSjopcOn0p)

**警告**:两个模型都没有接受过唱歌训练。如果你上传带有人声的音频，你可能会得到奇怪的结果。多乐器转录仍然不是一个完全解决的问题，所以你可能会得到奇怪的结果。

无论如何，我们希望你在转录过程中玩得开心!请随时在[@GoogleMagenta](https://twitter.com/googlemagenta)上发布任何有趣的输出…

###运行说明:

*确保使用GPU运行时，单击:__Runtime >>更改运行时类型>> GPU__
*按下每个单元格左边的▶️来执行单元格
*在__Load Model__单元格中，选择' ismir2021 '用于钢琴转录或' mt3 '用于多乐器转录
*在__Upload Audio__单元格中，当提示时从计算机中选择MP3或WAV文件
*使用__Transcribe Audio__单元格转录音频(可能需要几分钟，取决于音频的长度)

---

这款笔记本将基本使用数据发送到谷歌分析。欲了解更多信息，请参阅[谷歌的隐私政策](https://policies.google.com/privacy)。

In [ ]:
# 注释：此部分代码包含脚本的版权声明和设置环境的一些操作
# 该脚本的目的是安装 MT3（Magenta Transformer 3）及其依赖项，并设置环境

# 安装依赖项
!apt-get update -qq && apt-get install -qq libfluidsynth3 build-essential libasound2-dev libjack-dev

# 克隆 MT3 仓库
!git clone --branch=main https://github.com/magenta/mt3
!mv mt3 mt3_tmp; mv mt3_tmp/* .; rm -r mt3_tmp

# 使用 pip 安装 JAX、nest-asyncio 和 pyfluidsynth（指定版本）
!python3 -m pip install jax[cuda11_local] nest-asyncio pyfluidsynth==1.3.0 -e . -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# 复制检查点（模型参数）文件夹
!gsutil -q -m cp -r gs://mt3/checkpoints .

# 复制 soundfont 文件（音色文件，来源于 https://sites.google.com/site/soundfonts4u）
!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 .

# 导入 json 和 IPython 模块
import json
import IPython

# 以下两个函数（load_gtag 和 log_event）用于处理 Google Analytics 事件日志记录
# 日志记录是匿名的，仅存储音频和转录的一些基本统计信息，如音频长度和转录音符的数量

# 加载 gtag.js
def load_gtag():
    """Loads gtag.js."""
    # 注意：gtag.js 必须在执行合成的同一单元格中加载。它不会持续跨单元格执行！
    html_code = '''
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-4P250YRJ08"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());
      gtag('config', 'G-4P250YRJ08',
           {'referrer': document.referrer.split('?')[0],
            'anonymize_ip': true,
            'page_title': '',
            'page_referrer': '',
            'cookie_prefix': 'magenta',
            'cookie_domain': 'auto',
            'cookie_expires': 0,
            'cookie_flags': 'SameSite=None;Secure'});
    </script>
    '''
    IPython.display.display(IPython.display.HTML(html_code))

# 记录事件（匿名）
def log_event(event_name, event_details):
    """Log event with name and details dictionary."""
    details_json = json.dumps(event_details)
    js_string = "gtag('event', '%s', %s);" % (event_name, details_json)
    IPython.display.display(IPython.display.Javascript(js_string))

# 加载 gtag.js
load_gtag()

# 记录事件，事件名称为 'setupComplete'，详细信息为空字典
log_event('setupComplete', {})


In [ ]:
# 注释：导入必要的库和定义一些常量、函数等

# 导入库
import functools
import os
import numpy as np
import tensorflow.compat.v2 as tf

# 导入其他库和模块
import gin
import jax
import librosa
import note_seq
import seqio
import t5
import t5x

# 从 MT3（Magenta Transformer 3）模块中导入一些相关模块和工具
from mt3 import metrics_utils
from mt3 import models
from mt3 import network
from mt3 import note_sequences
from mt3 import preprocessors
from mt3 import spectrograms
from mt3 import vocabularies

# 从 Colab 中导入 files 模块
from google.colab import files

# 导入 nest_asyncio 模块，用于处理异步操作
import nest_asyncio
nest_asyncio.apply()

# 定义常量
SAMPLE_RATE = 16000
SF2_PATH = 'SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

# 定义函数 upload_audio 用于上传音频文件并转换为 samples
def upload_audio(sample_rate):
    # 从用户上传的文件中获取数据
    data = list(files.upload().values())
    if len(data) > 1:
        print('Multiple files uploaded; using only one.')
    # 将 WAV 数据转换为 samples
    return note_seq.audio_io.wav_data_to_samples_librosa(
        data[0], sample_rate=sample_rate)



class InferenceModel(object):
  """T5X 模型的音乐转录包装器。"""

  def __init__(self, checkpoint_path, model_type='mt3'):
    """
    初始化函数。

    参数:
    - checkpoint_path: 模型的检查点路径
    - model_type: 模型类型，可选值为 'ismir2021' 或 'mt3'

    """
    # 模型常量
    if model_type == 'ismir2021':
      num_velocity_bins = 127
      self.encoding_spec = note_sequences.NoteEncodingSpec
      self.inputs_length = 512
    elif model_type == 'mt3':
      num_velocity_bins = 1
      self.encoding_spec = note_sequences.NoteEncodingWithTiesSpec
      self.inputs_length = 256
    else:
      raise ValueError('unknown model_type: %s' % model_type)

    # Gin 文件路径
    gin_files = ['/content/mt3/gin/model.gin',
                 f'/content/mt3/gin/{model_type}.gin']

    # 模型参数
    self.batch_size = 8
    self.outputs_length = 1024
    self.sequence_length = {'inputs': self.inputs_length,
                            'targets': self.outputs_length}

    # PjitPartitioner
    self.partitioner = t5x.partitioning.PjitPartitioner(
        num_partitions=1)

    # 构建 Codecs 和 Vocabularies
    self.spectrogram_config = spectrograms.SpectrogramConfig()
    self.codec = vocabularies.build_codec(
        vocab_config=vocabularies.VocabularyConfig(
            num_velocity_bins=num_velocity_bins))
    self.vocabulary = vocabularies.vocabulary_from_codec(self.codec)
    self.output_features = {
        'inputs': seqio.ContinuousFeature(dtype=tf.float32, rank=2),
        'targets': seqio.Feature(vocabulary=self.vocabulary),
    }

    # 创建 T5X 模型
    self._parse_gin(gin_files)
    self.model = self._load_model()

    # 从检查点恢复
    self.restore_from_checkpoint(checkpoint_path)

  @property
  def input_shapes(self):
    """
    获取输入的形状。

    返回:
    - 输入的形状字典

    """
    return {
          'encoder_input_tokens': (self.batch_size, self.inputs_length),
          'decoder_input_tokens': (self.batch_size, self.outputs_length)
    }

  def _parse_gin(self, gin_files):
    """
    解析用于训练模型的 gin 文件。

    参数:
    - gin_files: 包含 gin 文件路径的列表

    """
    gin_bindings = [
        'from __gin__ import dynamic_registration',
        'from mt3 import vocabularies',
        'VOCAB_CONFIG=@vocabularies.VocabularyConfig()',
        'vocabularies.VocabularyConfig.num_velocity_bins=%NUM_VELOCITY_BINS'
    ]
    with gin.unlock_config():
      gin.parse_config_files_and_bindings(
          gin_files, gin_bindings, finalize_config=False)

  def _load_model(self):
    """
    在解析训练 gin 配置后加载 T5X `Model`。

    返回:
    - T5X `Model` 对象

    """
    model_config = gin.get_configurable(network.T5Config)()
    module = network.Transformer(config=model_config)
    return models.ContinuousInputsEncoderDecoderModel(
        module=module,
        input_vocabulary=self.output_features['inputs'].vocabulary,
        output_vocabulary=self.output_features['targets'].vocabulary,
        optimizer_def=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0),
        input_depth=spectrograms.input_depth(self.spectrogram_config))

  def restore_from_checkpoint(self, checkpoint_path):
    """
    从检查点中恢复训练状态，重置 self._predict_fn()。

    参数:
    - checkpoint_path: 模型检查点的路径

    """
    # 初始化训练状态
    train_state_initializer = t5x.utils.TrainStateInitializer(
      optimizer_def=self.model.optimizer_def,
      init_fn=self.model.get_initial_variables,
      input_shapes=self.input_shapes,
      partitioner=self.partitioner)

    # 恢复检查点的配置
    restore_checkpoint_cfg = t5x.utils.RestoreCheckpointConfig(
        path=checkpoint_path, mode='specific', dtype='float32')

    # 获取训练状态的轴
    train_state_axes = train_state_initializer.train_state_axes

    # 重置 self._predict_fn()
    self._predict_fn = self._get_predict_fn(train_state_axes)

    # 从检查点或初始状态恢复训练状态
    self._train_state = train_state_initializer.from_checkpoint_or_scratch(
        [restore_checkpoint_cfg], init_rng=jax.random.PRNGKey(0))

  @functools.lru_cache()
  def _get_predict_fn(self, train_state_axes):
    """
    生成一个用于解码的分区预测函数。

    参数:
    - train_state_axes: 训练状态的轴

    返回:
    - 分区预测函数

    """
    def partial_predict_fn(params, batch, decode_rng):
      return self.model.predict_batch_with_aux(
          params, batch, decoder_params={'decode_rng': None})
    return self.partitioner.partition(
        partial_predict_fn,
        in_axis_resources=(
            train_state_axes.params,
            t5x.partitioning.PartitionSpec('data',), None),
        out_axis_resources=t5x.partitioning.PartitionSpec('data',)
    )

  def predict_tokens(self, batch, seed=0):
    """
    从预处理的数据集批次中预测 tokens。

    参数:
    - batch: 预处理的数据集批次
    - seed: 随机数种子，默认为 0

    返回:
    - 预测的 tokens

    """
    # 调用分区预测函数进行预测
    prediction, _ = self._predict_fn(
        self._train_state.params, batch, jax.random.PRNGKey(seed))
    # 解码预测结果并返回
    return self.vocabulary.decode_tf(prediction).numpy()
  def __call__(self, audio):
    """
    从音频样本推断出音符序列。

    参数:
    - audio: 包含音频样本的 1 维 numpy 数组（16kHz），对于单个示例。

    返回:
    - 转录音频的音符序列。

    """
    # 将音频转换为 TF Dataset
    ds = self.audio_to_dataset(audio)
    # 对数据集进行预处理
    ds = self.preprocess(ds)

    # 将数据集转换为 T5X 模型输入
    model_ds = self.model.FEATURE_CONVERTER_CLS(pack=False)(
        ds, task_feature_lengths=self.sequence_length)
    model_ds = model_ds.batch(self.batch_size)

    # 获取推断结果的 tokens
    inferences = (tokens for batch in model_ds.as_numpy_iterator()
                  for tokens in self.predict_tokens(batch))

    # 处理预测结果
    predictions = []
    for example, tokens in zip(ds.as_numpy_iterator(), inferences):
      predictions.append(self.postprocess(tokens, example))

    # 将事件预测转换为 NoteSequence 对象
    result = metrics_utils.event_predictions_to_ns(
        predictions, codec=self.codec, encoding_spec=self.encoding_spec)
    return result['est_ns']

  def audio_to_dataset(self, audio):
    """
    从输入音频创建包含频谱图的 TF Dataset。

    参数:
    - audio: 输入音频的 1 维 numpy 数组

    返回:
    - 包含频谱图的 TF Dataset

    """
    # 将音频转换为频谱图数据集
    frames, frame_times = self._audio_to_frames(audio)
    return tf.data.Dataset.from_tensors({
        'inputs': frames,
        'input_times': frame_times,
    })

  def _audio_to_frames(self, audio):
    """
    从音频计算频谱图帧。

    参数:
    - audio: 输入音频的 1 维 numpy 数组

    返回:
    - 频谱图帧和对应的时间数组

    """
    # 计算频谱图帧
    frame_size = self.spectrogram_config.hop_width
    padding = [0, frame_size - len(audio) % frame_size]
    audio = np.pad(audio, padding, mode='constant')
    frames = spectrograms.split_audio(audio, self.spectrogram_config)
    num_frames = len(audio) // frame_size
    times = np.arange(num_frames) / self.spectrogram_config.frames_per_second
    return frames, times

  def preprocess(self, ds):
    """
    对数据集进行预处理。

    参数:
    - ds: TF Dataset

    返回:
    - 预处理后的 TF Dataset

    """
    # 预处理链
    pp_chain = [
        functools.partial(
            t5.data.preprocessors.split_tokens_to_inputs_length,
            sequence_length=self.sequence_length,
            output_features=self.output_features,
            feature_key='inputs',
            additional_feature_keys=['input_times']),
        # 在训练时发生缓存
        preprocessors.add_dummy_targets,
        functools.partial(
            preprocessors.compute_spectrograms,
            spectrogram_config=self.spectrogram_config)
    ]
    # 依次应用预处理步骤
    for pp in pp_chain:
      ds = pp(ds)
    return ds

  def postprocess(self, tokens, example):
    """
    后处理函数。

    参数:
    - tokens: 预测的 tokens
    - example: 示例

    返回:
    - 处理后的结果字典

    """
    # 去除 EOS（终止符）之后的 tokens
    tokens = self._trim_eos(tokens)
    start_time = example['input_times'][0]
    # 向下取整到最近的符号标记步长
    start_time -= start_time % (1 / self.codec.steps_per_second)
    return {
        'est_tokens': tokens,
        'start_time': start_time,
        # 内部 MT3 代码期望原始输入，这里未使用
        'raw_inputs': []
    }

  @staticmethod
  def _trim_eos(tokens):
    """
    去除 EOS（终止符）之后的 tokens。

    参数:
    - tokens: tokens 数组

    返回:
    - 去除 EOS 之后的 tokens

    """
    tokens = np.array(tokens, np.int32)
    if vocabularies.DECODED_EOS_ID in tokens:
      tokens = tokens[:np.argmax(tokens == vocabularies.DECODED_EOS_ID)]
    return tokens



In [ ]:
#@title 载入模型
#@markdown `ismir2021` 模型转录仅钢琴音符，包含音符速度。
#@markdown `mt3` 模型转录多个同时进行的乐器，但不包含音符速度。

MODEL = "mt3" #@param["ismir2021", "mt3"]

# 设置检查点路径
checkpoint_path = f'/content/checkpoints/{MODEL}/'

# 载入 Google Analytics 代码
load_gtag()

# 记录事件：模型加载开始
log_event('loadModelStart', {'event_category': MODEL})
# 创建 InferenceModel 实例
inference_model = InferenceModel(checkpoint_path, MODEL)
# 记录事件：模型加载完成
log_event('loadModelComplete', {'event_category': MODEL})


In [ ]:
#@title 上传音频

# 载入 Google Analytics 代码
load_gtag()

# 记录事件：音频上传开始
log_event('uploadAudioStart', {})
# 上传音频并获取音频数据
audio = upload_audio(sample_rate=SAMPLE_RATE)
# 记录事件：音频上传完成
log_event('uploadAudioComplete', {'value': round(len(audio) / SAMPLE_RATE)})

# 在Colab中播放音频
note_seq.notebook_utils.colab_play(audio, sample_rate=SAMPLE_RATE)


In [ ]:
#@title 转录音频
#@markdown 根据上传的音频文件的长度，这可能需要几分钟时间。

# 载入 Google Analytics 代码
load_gtag()

# 记录事件：转录开始
log_event('transcribeStart', {
    'event_category': MODEL,
    'value': round(len(audio) / SAMPLE_RATE)
})

# 使用 InferenceModel 进行音频转录
est_ns = inference_model(audio)

# 记录事件：转录完成
log_event('transcribeComplete', {
    'event_category': MODEL,
    'value': round(len(audio) / SAMPLE_RATE),
    'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
    'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
    'numPrograms': len(set(note.program for note in est_ns.notes
                           if not note.is_drum))
})

# 在Colab中播放转录结果
note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth,
                       sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
# 绘制转录结果的音符图
note_seq.plot_sequence(est_ns)


In [ ]:
#@title 下载 MIDI 转录

# 载入 Google Analytics 代码
load_gtag()

# 记录事件：下载转录文件
log_event('downloadTranscription', {
    'event_category': MODEL,
    'value': round(len(audio) / SAMPLE_RATE),
    'numNotes': sum(1 for note in est_ns.notes if not note.is_drum),
    'numDrumNotes': sum(1 for note in est_ns.notes if note.is_drum),
    'numPrograms': len(set(note.program for note in est_ns.notes
                           if not note.is_drum))
})

# 将 NoteSequence 转换为 MIDI 文件
note_seq.sequence_proto_to_midi_file(est_ns, '/tmp/transcribed.mid')
# 下载 MIDI 文件
files.download('/tmp/transcribed.mid')
